<a href="https://colab.research.google.com/github/darkfapper/Pandoras_Box/blob/main/Bayesian_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras.utils import to_categorical
import functools
tfk = tf.keras
tfd = tfp.distributions
tfb = tfp.bijectors
tfn = tfp.experimental.nn
tfpl = tfp.layers

NUM_CLASSES = 10
NUM_TRAIN_EXAMPLES = 100



# Load in the data
cifar10 = tf.keras.datasets.cifar10

# Distribute it to train and test set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train/ 255.
x_test = x_test / 255.

y_train_oh = to_categorical(y_train, NUM_CLASSES)
y_test_oh = to_categorical(y_test, NUM_CLASSES)

# x_train =np.expand_dims(x_train, -1).astype("float32") / 255.
# x_test =np.expand_dims(x_test, -1).astype("float32") / 255.


batch_size = 32
learning_rate = 0.0001
epochs = 150
num_monte_carlo = 50

BayesConv2D = functools.partial(tfn.ConvolutionVariationalFlipout,penalty_weight=1. / x_train.shape[0],rank = 2)
BayesDense = functools.partial(tfn.AffineVariationalFlipout,penalty_weight=1. / x_train.shape[0])

tf.experimental.numpy.experimental_enable_numpy_behavior()
kl_divergence_function = (lambda q, p, _: tfd.kl_divergence(q, p) /  tf.cast(x_train.shape[0], dtype=tf.float32))

def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = tf.keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model

INPUT_SHAPE = (32,32,3)
KERNEL_SIZE = (3,3)

def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size

    # nvp = tfd.TransformedDistribution(
    # distribution=tfd.MultivariateNormalDiag(n),
    # bijector=tfb.RealNVP(
    #     num_masked=2,
    #     shift_and_log_scale_fn=tfb.real_nvp_default_template(
    #         hidden_layers=[128, 128])))

    # posterior_model = tf.keras.Sequential(
    #     [
    #         tfp.layers.VariableLayer(
    #             tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
    #         ),
    #         nvp,
    #     ]
    posterior_model = tfk.Sequential([
    # NOTE: This model takes no input and outputs a Distribution.  (We use
    # the batch_size and type of the input, but there are no actual input
    # values because the last dimension of the shape is 0.)
    #
    # For conditional density estimation, the model would take the
    # conditioning values as input.)
    tfk.layers.InputLayer(input_shape=(0,), dtype=tf.float32),

    # Given the empty input, return a standard normal distribution with
    # matching batch_shape and event_shape of [2].
    tfpl.DistributionLambda(lambda t: tfd.MultivariateNormalDiag(
        # pylint: disable=g-long-lambda
        loc=tf.zeros(tf.concat([tf.shape(t)[:-1], [2]], axis=0)),
        scale_diag=[1., 1.])),

    # Transform the standard normal distribution with event_shape of [2] to
    # the target distribution with event_shape of [2].
    tfpl.AutoregressiveTransform(tfb.AutoregressiveNetwork(
        params=2, hidden_units=[10], activation='relu'))])
    return posterior_model

def get_conv_bayes(filters:int,activation=tf.nn.selu):

    return tfp.layers.Convolution2DFlipout(filters=filters, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE,padding = "same",
                                           bias_divergence_fn=kl_divergence_function,kernel_divergence_fn=kl_divergence_function)


def get_dense_bayes(units:int,activation=tf.nn.selu):
    return tfp.layers.DenseFlipout(units=units, activation=activation,bias_divergence_fn=kl_divergence_function,kernel_divergence_fn=kl_divergence_function)

170498071/170498071 [==============================] - 6s 0us/step


In [ ]:

model = Sequential()

# Convolutional Layer
model.add(get_conv_bayes(32))
model.add(BatchNormalization())
model.add(get_conv_bayes(32))
model.add(BatchNormalization())
# Pooling layer
model.add(MaxPool2D(pool_size=(2, 2)))



model.add(get_conv_bayes(64))
model.add(BatchNormalization())
model.add(get_conv_bayes(64))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(get_conv_bayes(128))
model.add(BatchNormalization())
model.add(get_conv_bayes(128))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(get_conv_bayes(256))
model.add(BatchNormalization())
model.add(get_conv_bayes(256))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

# model.add(get_conv_bayes(512))
# model.add(BatchNormalization())
# model.add(get_conv_bayes(512))
# model.add(BatchNormalization())
# model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(get_dense_bayes(1024))
model.add(BatchNormalization())
# model.add(get_dense_bayes(256))
# model.add(BatchNormalization())
model.add(get_dense_bayes(10, activation='softmax'))

METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=METRICS)

/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/layers/util.py:98: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  loc = add_variable_fn(
/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/layers/util.py:108: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  untransformed_scale = add_variable_fn(


In [ ]:
# INPUT_SHAPE = (32, 32, 3)
# KERNEL_SIZE = (3, 3)
# model = Sequential()

# # Convolutional Layer
# model.add(Conv2D(filters=32, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
# model.add(BatchNormalization())
# model.add(Conv2D(filters=32, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
# model.add(BatchNormalization())
# # Pooling layer
# model.add(MaxPool2D(pool_size=(2, 2)))
# # Dropout layers
# # model.add(Dropout(0.25))


# model.add(Conv2D(filters=64, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
# model.add(BatchNormalization())
# model.add(Conv2D(filters=64, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
# model.add(BatchNormalization())
# model.add(MaxPool2D(pool_size=(2, 2)))
# # model.add(Dropout(0.25))

# model.add(Conv2D(filters=128, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
# model.add(BatchNormalization())
# model.add(Conv2D(filters=128, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
# model.add(BatchNormalization())
# model.add(MaxPool2D(pool_size=(2, 2)))

# model.add(Conv2D(filters=256, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
# model.add(BatchNormalization())
# model.add(Conv2D(filters=256, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
# model.add(BatchNormalization())
# model.add(MaxPool2D(pool_size=(2, 2)))
# # model.add(Dropout(0.25))

# model.add(Flatten())
# # model.add(Dropout(0.2))
# model.add(Dense(1024, activation='relu'))
# # model.add(Dropout(0.25))
# model.add(Dense(10, activation='softmax'))

# METRICS = [
#     'accuracy',
#     tf.keras.metrics.Precision(name='precision'),
#     tf.keras.metrics.Recall(name='recall')
# ]
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=METRICS)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_flipout (Conv2DFlip  (None, 32, 32, 32)        1760      
 out)                                                            
                                                                 
 batch_normalization (Batch  (None, 32, 32, 32)        128       
 Normalization)                                                  
                                                                 
 conv2d_flipout_1 (Conv2DFl  (None, 32, 32, 32)        18464     
 ipout)                                                          
                                                                 
 batch_normalization_1 (Bat  (None, 32, 32, 32)        128       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 32)        0

In [ ]:
model.fit(x_train, y_train_oh, epochs=150,validation_data=(x_test, y_test_oh))

Epoch 1/150
1563/1563 [==============================] - 390s 244ms/step - loss: 112.9078 - accuracy: 0.2911 - precision: 0.3360 - recall: 0.2049 - val_loss: 108.5156 - val_accuracy: 0.3591 - val_precision: 0.4104 - val_recall: 0.2642
Epoch 2/150
1563/1563 [==============================] - 379s 242ms/step - loss: 102.2154 - accuracy: 0.4100 - precision: 0.4895 - recall: 0.3090 - val_loss: 95.4231 - val_accuracy: 0.4343 - val_precision: 0.5075 - val_recall: 0.3380
Epoch 3/150
1563/1563 [==============================] - 360s 230ms/step - loss: 87.5359 - accuracy: 0.4888 - precision: 0.5750 - recall: 0.3898 - val_loss: 79.8049 - val_accuracy: 0.5257 - val_precision: 0.6031 - val_recall: 0.4411
Epoch 4/150
1563/1563 [==============================] - 360s 231ms/step - loss: 72.3127 - accuracy: 0.5503 - precision: 0.6377 - recall: 0.4583 - val_loss: 65.2576 - val_accuracy: 0.5549 - val_precision: 0.6426 - val_recall: 0.4653
Epoch 5/150
1563/1563 [==============================] - 358s 229

In [ ]:
model.evaluate(x_test, y_test_oh)

In [ ]:
lista = []
for i in range(num_monte_carlo):
  lista.append(model(x_test))

In [ ]:
import matplotlib.pyplot as plt
arr = np.array(lista)

x = np.random.randint(0, x_test.shape[0])
val = arr[:,x,:]
img = x_test[x,:]

label = y_test[x]
#plt.figure(figsize=(10,10))
print(f"Label: {label}")
plt.imshow(img)
val = pd.DataFrame(val)
val.hist( bins=25,
grid=False, figsize=(15,12), color='#86bf91', zorder=2, rwidth=0.9)

In [ ]:
## Texte Com Imagem False
img_random = np.random.randint(0, 255, size=(32,32,3))/255.
model.predict(img_random.reshape(1,32,32,3))
plt.imshow(img_random)
val = pd.DataFrame(val)
val.hist( bins=25,
grid=False, figsize=(15,12), color='#86bf91', zorder=2, rwidth=0.9)

In [ ]:
from PIL import Image
img = Image.fromarray(x_train[999,:],"RGB")
img.save("img.png")
img.show()